In [2]:
import glob
import pandas as pd
# import tensorflow as tf
import os
from typing import Generator
from datetime import datetime
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [3]:
def txt_file_generator(file_list: list) -> Generator[pd.DataFrame, None, None]:
    for txt_file in file_list:
        yield read_txt_file(txt_file)

In [4]:
def find_txt_files(directory):
    txt_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):    
#                 print(os.path.join(root, file))
                txt_files.append(os.path.join(root, file))
    return txt_files

In [5]:
def read_txt_file(file_path):
    # 从文件名中提取年、月、日
    date_str = os.path.basename(file_path).split('.')[0] # 例如，得到"20100101"
    year = int(date_str[:4])
    month = int(date_str[4:6])
    day = int(date_str[6:])

    df = pd.read_csv(file_path, delim_whitespace=True, header=None)
    
    # 将年、月、日添加为新列
    df['year'] = year
    df['month'] = month
    df['day'] = day

    return df


火险指数
---

In [14]:
# txt_files = find_txt_files(r"E:\dataset\model\China")

# dataframes = []  
# for txt_file in txt_files:
#     df = read_txt_file(txt_file)
#     dataframes.append(df)
# df_CHINA = pd.concat(dataframes)  
# df_CHINA.columns =['sta','lon','lat','year','month','day','FFDI']
# df_CHINA.to_csv(r'E:\fire_dataset\model\CHINA.csv')

In [15]:
# df_CHINA.head()

,sta,lon,lat,year,month,day,FFDI
0,50136,122.52,52.97,2010,1,1,0.0
1,50246,124.72,52.35,2010,1,1,0.0
2,50247,123.57,52.03,2010,1,1,0.0
3,50349,124.33,51.70,2010,1,1,0.0
4,50353,126.65,51.72,2010,1,1,0.0


美国模型
---

In [22]:
txt_files = find_txt_files(r"E:\dataset\model\USA")

df_gen = txt_file_generator(txt_files)
df_USA = pd.concat(df_gen, axis=0)

df_USA.columns =['sta','lon','lat','yth1','yth10','yth100','yth1000','kb','erc','sc','bi','ic','p','year','month','day']    
df_USA.to_csv(r'E:\fire_dataset\model\USA.csv')

In [23]:
df_USA.head()

,sta,lon,lat,yth1,yth10,yth100,yth1000,kb,erc,sc,bi,ic,p,year,month,day
0,50136,122.52,52.97,13.51,15.71,14.12,15.02,12.41,0.0,0.0,0.0,0.0,0.29,2010,1,1
1,50246,124.72,52.35,12.48,14.62,13.72,13.73,66.64,0.0,0.0,0.0,0.0,0.40,2010,1,1
2,50247,123.57,52.03,13.43,15.52,13.42,14.66,29.20,0.0,0.0,0.0,0.0,0.32,2010,1,1
3,50349,124.33,51.70,13.52,15.62,13.47,14.31,21.04,0.0,0.0,0.0,0.0,0.30,2010,1,1
4,50353,126.65,51.72,12.43,14.59,13.86,13.80,93.10,0.0,0.0,0.0,0.0,0.46,2010,1,1


加拿大
---

In [27]:
txt_files = find_txt_files(r'E:\dataset\model\Canada')

df_gen = txt_file_generator(txt_files)
df_CANADA = pd.concat(df_gen, axis=0)

df_CANADA.columns =['sta','lon','lat','FFMC','DMC','DC','FWI','ISI','BUI','DSR','year','month','day']
df_CANADA.to_csv(r'E:\fire_dataset\model\CANADA.csv')

匹配
---

In [17]:
df_USA = pd.read_csv(r'E:\fire_dataset\model\USA.csv', index_col=0)
df_CANADA = pd.read_csv(r'E:\fire_dataset\model\CANADA.csv',index_col=0)
df_CHINA = pd.read_csv(r'E:\fire_dataset\model\China.csv', index_col=0)

# df_USA = df_USA.drop_duplicates(subset= ['sta'],keep='first',inplace=False)
# df_CANADA = df_CANADA.drop_duplicates(subset= ['sta'],keep='first',inplace=False)
merged_uc = df_USA.merge(df_CANADA, on=['sta', 'lon', 'lat','year','month','day'])

C:\Users\HTHT\AppData\Local\Temp\ipykernel_4916\792661640.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_USA = pd.read_csv(r'E:\fire_dataset\model\USA.csv', index_col=0)


In [22]:
# merged_uc = merged_uc.drop(columns=['lon', 'lat'])
merged_all =merged_uc.merge(df_CHINA, on = ['sta','year','month','day','lat','lon'])
# merged_all = merged_all.drop_duplicates(subset= ['sta'],keep='first',inplace=False)

In [23]:
merged_all

,sta,lon,lat,yth1,yth10,yth100,yth1000,kb,erc,sc,...,month,day,FFMC,DMC,DC,FWI,ISI,BUI,DSR,FFDI
0,50136,122.52,52.97,13.51,15.71,14.12,15.02,12.41,0.00,0.00,...,1,1,52.06,4.49,131.07,0.16,0.29,8.27,0.00,0.00
1,50246,124.72,52.35,12.48,14.62,13.72,13.73,66.64,0.00,0.00,...,1,1,65.71,15.97,145.25,0.88,0.84,25.05,0.02,0.00
2,50247,123.57,52.03,13.43,15.52,13.42,14.66,29.20,0.00,0.00,...,1,1,51.89,10.28,87.69,0.20,0.26,15.90,0.00,0.00
3,50349,124.33,51.70,13.52,15.62,13.47,14.31,21.04,0.00,0.00,...,1,1,62.15,7.98,97.72,0.48,0.68,13.25,0.01,0.00
4,50353,126.65,51.72,12.43,14.59,13.86,13.80,93.10,0.00,0.00,...,1,1,64.51,15.32,214.37,0.94,0.87,26.00,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6484751,59941,109.52,18.77,12.59,15.24,16.86,15.76,478.81,7.83,1.21,...,12,31,86.65,52.35,209.80,15.73,5.19,64.48,3.57,0.55
6484752,59945,109.70,18.65,11.37,14.02,16.86,14.55,465.05,9.58,1.23,...,12,31,88.02,66.89,185.90,18.11,5.88,70.43,4.58,0.58
6484753,59948,109.59,18.22,12.65,15.57,18.59,17.21,298.79,5.58,6.43,...,12,31,84.89,24.36,144.26,46.48,39.19,34.26,24.30,0.70
6484754,59951,110.33,18.80,14.26,17.49,20.55,20.90,21.22,2.64,0.34,...,12,31,81.55,2.23,11.60,1.35,3.05,3.02,0.05,0.25


In [24]:
merged_all.to_csv(r'E:\fire_dataset\model\model.csv')

In [25]:
merged_all.head(100).to_csv(r'E:\fire_dataset\model\model100.csv')


final_dataset
___

In [20]:
model = pd.read_csv('model.csv', index_col=0)
obs = pd.read_csv('obs.csv', index_col=0)

columns_to_drop = ['lat', 'lon']
obs.drop(columns=columns_to_drop, inplace=True)
columns_to_rename = {
    'Year': 'year',
    'Mon': 'month',
    'Day': 'day'
}

# 使用rename方法来重命名列
obs.rename(columns=columns_to_rename, inplace=True)

C:\Users\HTHT\AppData\Local\Temp\ipykernel_15668\1377619108.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  model = pd.read_csv('model.csv', index_col=0)


In [21]:

merge = obs.merge(model,on = ['sta','year','month','day'])
merge.to_csv('dataset.csv', index=None)

In [22]:
merge

,sta,Alti,year,month,day,TEM_Max,TEM_Min,RHU_Min,PRE_Time_2020,Snow_Depth,...,ic,p,FFMC,DMC,DC,FWI,ISI,BUI,DSR,FFDI
0,50136,433.0,2010,1,1,-25.4,-38.5,61,0.6,25.0,...,0.0,0.29,52.06,4.49,131.07,0.16,0.29,8.27,0.00,0.00
1,50246,361.9,2010,1,1,-21.9,-33.0,56,0.7,22.0,...,0.0,0.40,65.71,15.97,145.25,0.88,0.84,25.05,0.02,0.00
2,50247,514.5,2010,1,1,-24.1,-40.5,61,1.5,15.0,...,0.0,0.32,51.89,10.28,87.69,0.20,0.26,15.90,0.00,0.00
3,50349,494.6,2010,1,1,-21.0,-37.2,62,0.8,14.0,...,0.0,0.30,62.15,7.98,97.72,0.48,0.68,13.25,0.01,0.00
4,50353,177.4,2010,1,1,-20.2,-26.5,56,0.6,15.0,...,0.0,0.46,64.51,15.32,214.37,0.94,0.87,26.00,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6484755,59941,328.5,2017,12,31,20.8,15.6,65,0.0,0.0,...,5.0,0.96,86.65,52.35,209.80,15.73,5.19,64.48,3.57,0.55
6484756,59945,68.6,2017,12,31,23.3,19.2,59,0.0,0.0,...,7.0,0.96,88.02,66.89,185.90,18.11,5.88,70.43,4.58,0.58
6484757,59948,419.4,2017,12,31,21.1,16.9,65,0.0,0.0,...,12.0,0.83,84.89,24.36,144.26,46.48,39.19,34.26,24.30,0.70
6484758,59951,39.9,2017,12,31,21.2,18.2,71,0.0,0.0,...,1.0,0.30,81.55,2.23,11.60,1.35,3.05,3.02,0.05,0.25


In [6]:
pd_100 = pd.read_csv(r'dataset.csv', nrows=100)
pd_100.to_csv(r'dataset100.csv')